# Sprint13 tensorflow Memo

In [9]:
!/Users/yoshiko/.pyenv/versions/anaconda3-5.3.1/bin/python -m pip install tensorflow==1.14.0

In [8]:
import sys
sys.executable

'/Users/yoshiko/.pyenv/versions/anaconda3-5.3.1/bin/python'

In [8]:
import sys
import numpy as np
import tensorflow as tf
from keras import backend as K 

In [9]:
"""
バージョンチェック
"""

print(tf.VERSION)  # tf.__version__でもよい

1.14.0


## tf.Session()

tf.Session()は、`with`構文の中で、セッションオブジェクトとして引数に取ったOperation(値を出力するノード)を実行する。

常にセッションオブジェクトを参照する必要があるため、non-interactiveである。

データフローグラフとは独立して定義される。

https://www.tensorflow.org/guide/graphs#using_tfsessionrun_to_execute_operations

In [10]:
"""

tf.Session.runとfeed_dictについて

tf.Session.runは、オプションでfeed_dictのディクショナリを受け取る。
これはつまり、tf.Tensorオブジェクト（通常はtf.placeholderテンソル）から
値（通常はPythonスカラー、リスト、またはNumPy配列）への
マッピングであり、実行中のテンソルの代わりとなる。


"""



# ベースライン

"""

# placeholder　：three floating-point valuesのベクトルが入ってくると予想される
# y　　　　　　　：placeholderに依存した計算

"""
    
    
x = tf.placeholder(tf.float32, shape=(3))
y = tf.square(x)


with tf.Session() as sess:
    
    # a valueをFeedするたびに、`y`を評価する際に返ってくる結果が変わる。 
    
    print(sess.run(y, feed_dict={x: [1.0, 2.0, 3.0]}))  # => "[1.0, 4.0, 9.0]"
    print(sess.run(y, feed_dict={x: [0.0, 0.0, 5.0]}))  # => "[0.0, 0.0, 25.0]"

    
    """
    Raises `tf.errors.InvalidArgumentError`
    """
    # sess.run(y)

    """
    Raises `ValueError`
    """
    # sess.run(y, {x: 37.0})
    
    

[1. 4. 9.]
[ 0.  0. 25.]


### tf.initialize_all_variables() と tf.global_variables_initializer()


tf.initialize_all_variables()は、2017-03-02以降に廃止され、 tf.global_variables_initializer()に移行した。

`Warning: THIS FUNCTION IS DEPRECATED. It will be removed after 2017-03-02. Instructions for updating: Use tf.global_variables_initializer instead.`

https://www.tensorflow.org/api_docs/python/tf/initialize_all_variables



`参考までに`

さらに、version 2.0では、tf.Sessionを用いないため、tf.global_variables_initializer()も使う必要がない。

"How do I initialize tf.global_variables_initializer() with version 2.0?"

https://www.reddit.com/r/tensorflow/comments/bk94lj/how_do_i_initialize_tfglobal_variables/

In [4]:
# tf.global_variables_initializer()によって変数が初期値(今回は0)で初期化される


State = tf.Variable(0, name="counter")
one = tf.constant(1)
new_value = tf.add(State, one)
update = tf.assign(State, new_value) 

init_op = tf.global_variables_initializer() 


# Sessionインスタンス
sess = tf.Session()


# 初期化前にはStateオペレーションを実行できない
"""
Raises `FailedPreconditionError`：Attempting to use uninitialized value
"""
# print(sess.run(State))


# initialize
sess.run(init_op)


# 初期値(今回は0)がStateオペレーションから返される     # 0
print("Stateの初期値：{}".format(sess.run(State)))


# Stateに値を代入するオペレーションupdateを実行
sess.run(update)

# Stateオペレーションの再実行
print(sess.run(State))                          # 1

Stateの初期値：0
1


### initialized_value()

tf.global_variables_initializer()は、変数が初期化される順序は指定されていない。
ゆえに、変数の初期値が別の変数の値に依存している場合、エラーが発生する可能性がある。
すべての変数が初期化されていないコンテキストで変数の値を使用するときはいつでも（たとえば、別の変数を初期化するときに変数の値を使用する場合

デフォルトでtf.global_variables_initializerは、変数が初期化される順序は指定されていないことに注意してください。したがって、変数の初期値が別の変数の値に依存している場合、エラーが発生する可能性があります。すべての変数が初期化されていないコンテキストで変数の値を使用するときはいつでも（たとえば、別の変数を初期化するときに変数の値を使用する場合）、variable.initialized_value()代わりに使用するのが最善です 

## tf.InteractiveSession()


tf.InteractiveSession()はオペレーションを明示的にセッションオブジェクトに渡す必要がなく、interactiveに操作可能である。

データフローグラフを構築する前にインスタンスを作成する必要がある。

`sess.close()`を忘れずに。

https://stackoverflow.com/questions/41791469/whats-the-difference-between-tf-session-and-tf-interactivesession

## eval() を使ってみよう


tf.InteractiveSession()と組み合わせてみよう

In [11]:
"""
tf.reduce_sum()：配列の要素を全て足す
"""


# もしも eval()がなかったら

sess = tf.InteractiveSession()

x = tf.constant([[1, 1, 1], [1, 1, 1]])
print(tf.reduce_sum(x))
print(tf.reduce_sum(x, 0))
print(tf.reduce_sum(x, 1))
print(tf.reduce_sum(x, 1, keepdims=True))
print(tf.reduce_sum(x, [0, 1]))

sess.close()

Tensor("Sum_15:0", shape=(), dtype=int32)
Tensor("Sum_16:0", shape=(3,), dtype=int32)
Tensor("Sum_17:0", shape=(2,), dtype=int32)
Tensor("Sum_18:0", shape=(2, 1), dtype=int32)
Tensor("Sum_19:0", shape=(), dtype=int32)


In [12]:


# もしも eval()があったら

sess = tf.InteractiveSession()

x = tf.constant([[1, 1, 1], [1, 1, 1]])
print(tf.reduce_sum(x).eval())                    # 6
print(tf.reduce_sum(x, 0).eval())                 # [2, 2, 2]
print(tf.reduce_sum(x, 1).eval())                 # [3, 3]
print(tf.reduce_sum(x, 1, keepdims=True).eval())  # [[3], [3]]
print(tf.reduce_sum(x, [0, 1]).eval())            # 6

sess.close()

6
[2 2 2]
[3 3]
[[3]
 [3]]
6


## keras.backend.eval()

`おまけ`

先ほどのtf.InteractiveSession()とeval()の組み合わせの代わりにkeras.backend.eval()でも値が取り出せる。

In [13]:
"""
from keras import backend as K
"""


# tf.InteractiveSession()は不要


x = tf.constant([[1, 1, 1], [1, 1, 1]])
print(K.eval(tf.reduce_sum(x)))                    # 6
print(K.eval(tf.reduce_sum(x, 0)))                 # [2, 2, 2]
print(K.eval(tf.reduce_sum(x, 1)))                 # [3, 3]
print(K.eval(tf.reduce_sum(x, 1, keepdims=True)))  # [[3], [3]]
print(K.eval(tf.reduce_sum(x, [0, 1])))            # 6

6
[2 2 2]
[3 3]
[[3]
 [3]]
6
